In [23]:

"""
╔═══════════════════════════════════════════════════════════════════════════╗
║           GEMINI ADK MULTI-AGENT NUTRITION SYSTEM (Production)      ║
║                                                                     ║
║  Built with Google's Official Agent Development Kit (ADK-Python)    ║
║  Demonstrating:                                                     ║
║  • ADK Agent Registration & Orchestration                           ║
║  • Built-in Session & Context Management                            ║
║  • Native Observability & Tracing                                   ║
║  • Production-Ready Error Handling                                  ║
║  • Sequential Multi-Agent Workflow (Analyst → Coach)                ║
╚═══════════════════════════════════════════════════════════════════════════╝
"""

"\n╔═══════════════════════════════════════════════════════════════════════════╗\n║           GEMINI ADK MULTI-AGENT NUTRITION SYSTEM (Production)      ║\n║                                                                     ║\n║  Built with Google's Official Agent Development Kit (ADK-Python)    ║\n║  Demonstrating:                                                     ║\n║  • ADK Agent Registration & Orchestration                           ║\n║  • Built-in Session & Context Management                            ║\n║  • Native Observability & Tracing                                   ║\n║  • Production-Ready Error Handling                                  ║\n║  • Sequential Multi-Agent Workflow (Analyst → Coach)                ║\n╚═══════════════════════════════════════════════════════════════════════════╝\n"

In [24]:
import os
from datetime import datetime
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
import json

# ADK Imports
from google import genai
from google.genai import types
from google.genai.types import Tool, FunctionDeclaration, Schema

# Visualization
import pandas as pd
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.progress import Progress, SpinnerColumn, TextColumn
from rich.tree import Tree
from rich import box
from rich.layout import Layout
from rich.live import Live

In [25]:
# CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════

GOOGLE_API_KEY = "YOUR-API-KEY"
MODEL_NAME = "gemini-2.5-flash"

console = Console()

In [26]:
# DATA MODELS


@dataclass
class UserProfile:
    """User health profile for personalized recommendations."""
    weight_kg: float
    height_cm: float
    age: int
    sex: str
    activity_level: str = "moderate"
    goal: str = "maintain"
    
    def to_dict(self) -> Dict:
        return {
            "weight_kg": self.weight_kg,
            "height_cm": self.height_cm,
            "age": self.age,
            "sex": self.sex,
            "activity_level": self.activity_level,
            "goal": self.goal
        }

@dataclass
class NutritionAnalysis:
    """Results from nutritional analysis."""
    total_calories: int = 0
    total_protein: int = 0
    total_carbs: int = 0
    total_fats: int = 0
    foods: List[Dict[str, Any]] = field(default_factory=list)
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class AgentTrace:
    """Trace information for observability."""
    agent_name: str
    start_time: float
    end_time: Optional[float] = None
    status: str = "running"
    tool_calls: List[str] = field(default_factory=list)
    output: Optional[str] = None
    
    @property
    def duration(self) -> float:
        if self.end_time:
            return round(self.end_time - self.start_time, 3)
        return 0.0


In [27]:
# NUTRITION DATABASE


NUTRITION_DB = pd.DataFrame([
    ["wheat_roti", 250, 8, 50, 2, "Grains"],
    ["white_rice", 130, 2, 28, 0, "Grains"],
    ["brown_rice", 112, 2, 24, 1, "Grains"],
    ["dal_cooked", 105, 8, 14, 0, "Legumes"],
    ["paneer", 265, 18, 1, 20, "Dairy"],
    ["egg_whole", 155, 13, 1, 11, "Protein"],
    ["chicken_breast", 165, 31, 0, 3, "Protein"],
    ["banana", 89, 1, 23, 0, "Fruits"],
    ["apple", 52, 0, 14, 0, "Fruits"],
    ["milk_whole", 60, 3, 5, 3, "Dairy"],
    ["almonds", 579, 21, 22, 50, "Nuts"],
    ["spinach", 23, 3, 4, 0, "Vegetables"],
], columns=["food", "calories", "protein", "carbs", "fat", "category"])

In [28]:
# ADK TOOL DEFINITIONS (Function Declarations)


def get_food_nutrition_impl(food_name: str) -> Dict[str, Any]:
    """Implementation of food nutrition lookup."""
    from difflib import get_close_matches
    
    matches = get_close_matches(
        food_name.lower(), 
        NUTRITION_DB['food'].tolist(), 
        n=1, 
        cutoff=0.5
    )
    
    if matches:
        result = NUTRITION_DB[NUTRITION_DB['food'] == matches[0]].iloc[0].to_dict()
        return {
            "found": True,
            "food": matches[0],
            "calories": int(result['calories']),
            "protein": int(result['protein']),
            "carbs": int(result['carbs']),
            "fat": int(result['fat']),
            "category": result['category']
        }
    
    return {"found": False, "error": f"Food '{food_name}' not found in database"}


def calculate_tdee_impl(
    weight_kg: float,
    height_cm: float,
    age: int,
    sex: str,
    activity_level: str = "moderate"
) -> Dict[str, Any]:
    """Implementation of TDEE calculation."""
    # Mifflin-St Jeor equation
    bmr = (10 * weight_kg) + (6.25 * height_cm) - (5 * age)
    bmr = bmr + 5 if sex.lower() == 'male' else bmr - 161
    
    activity_multipliers = {
        "sedentary": 1.2,
        "light": 1.375,
        "moderate": 1.55,
        "active": 1.725,
        "very_active": 1.9
    }
    
    multiplier = activity_multipliers.get(activity_level.lower(), 1.55)
    tdee = int(bmr * multiplier)
    
    return {
        "bmr": int(bmr),
        "tdee": tdee,
        "activity_level": activity_level,
        "multiplier": multiplier,
        "maintenance_calories": tdee
    }


# ADK Function Declarations
get_food_nutrition_declaration = FunctionDeclaration(
    name="get_food_nutrition",
    description="Retrieves nutritional information for a specific food item from the database. Use fuzzy matching to find similar foods.",
    parameters={
        "type": "object",
        "properties": {
            "food_name": {
                "type": "string",
                "description": "Name of the food item to look up (e.g., 'chicken breast', 'banana', 'rice')"
            }
        },
        "required": ["food_name"]
    }
)

calculate_tdee_declaration = FunctionDeclaration(
    name="calculate_tdee",
    description="Calculates Total Daily Energy Expenditure (TDEE) using the Mifflin-St Jeor equation. Returns daily calorie needs based on user profile.",
    parameters={
        "type": "object",
        "properties": {
            "weight_kg": {"type": "number", "description": "Body weight in kilograms"},
            "height_cm": {"type": "number", "description": "Height in centimeters"},
            "age": {"type": "integer", "description": "Age in years"},
            "sex": {"type": "string", "description": "Biological sex: 'male' or 'female'"},
            "activity_level": {
                "type": "string",
                "description": "Activity level: 'sedentary', 'light', 'moderate', 'active', or 'very_active'",
                "default": "moderate"
            }
        },
        "required": ["weight_kg", "height_cm", "age", "sex"]
    }
)

# ADK Tool Registration
nutrition_tools = Tool(function_declarations=[
    get_food_nutrition_declaration,
    calculate_tdee_declaration
])


In [29]:
# ADK AGENT SYSTEM


class ADKNutritionSystem:
    """
    Production-grade multi-agent system using Google ADK.
    Implements sequential agent workflow with built-in observability.
    """
    
    def __init__(self, api_key: str):
        self.client = genai.Client(api_key=api_key)
        self.traces: List[AgentTrace] = []
        
        console.print("[green]✓[/green] ADK Client initialized")
        console.print(f"[green]✓[/green] Model: {MODEL_NAME}")
        console.print(f"[green]✓[/green] Tools registered: {len(nutrition_tools.function_declarations)}")
    
    def _execute_tool_call(self, tool_name: str, args: Dict) -> Any:
        """Execute tool calls with proper routing."""
        if tool_name == "get_food_nutrition":
            return get_food_nutrition_impl(**args)
        elif tool_name == "calculate_tdee":
            return calculate_tdee_impl(**args)
        else:
            return {"error": f"Unknown tool: {tool_name}"}
    
    def run_analyst_agent(self, meal_description: str) -> NutritionAnalysis:
        """
        AGENT 1: Nutrition Analyst
        Uses ADK to orchestrate food analysis with automatic tool calling.
        """
        import time
        trace = AgentTrace(agent_name="NutritionAnalyst", start_time=time.time())
        
        console.print("\n[bold cyan]🔬 Analyst Agent: Starting Analysis[/bold cyan]")
        
        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            console=console,
            transient=True
        ) as progress:
            progress.add_task("Analyzing meal components...", total=None)
            
            system_instruction = """You are a Nutrition Analyst AI specialized in food analysis.
Your task is to:
1. Parse the meal description carefully
2. Identify ALL food items mentioned
3. Use get_food_nutrition() for EACH food item
4. Calculate accurate totals
5. Return ONLY valid JSON with no markdown

Output format:
{
  "total_calories": <number>,
  "total_protein": <number>,
  "total_carbs": <number>,
  "total_fats": <number>,
  "foods": [{"name": "<food>", "calories": <n>, "protein": <n>, "carbs": <n>, "fat": <n>}]
}"""
            
            messages = [
                types.Content(
                    role="user",
                    parts=[types.Part(text=f"Analyze this meal: {meal_description}")]
                )
            ]
            
            # ADK Agent execution with tools
            response = self.client.models.generate_content(
                model=MODEL_NAME,
                contents=messages,
                config=types.GenerateContentConfig(
                    system_instruction=system_instruction,
                    tools=[nutrition_tools],
                    temperature=0.1
                )
            )
            
            # Process tool calls if any
            final_response = response
            for candidate in response.candidates:
                for part in candidate.content.parts:
                    if hasattr(part, 'function_call') and part.function_call:
                        fc = part.function_call
                        trace.tool_calls.append(f"{fc.name}({fc.args})")
                        
                        # Execute tool
                        tool_result = self._execute_tool_call(fc.name, dict(fc.args))
                        
                        # Continue conversation with tool result
                        messages.append(candidate.content)
                        messages.append(types.Content(
                            role="user",
                            parts=[types.Part(
                                function_response=types.FunctionResponse(
                                    name=fc.name,
                                    response=tool_result
                                )
                            )]
                        ))
                        
                        # Get final response
                        final_response = self.client.models.generate_content(
                            model=MODEL_NAME,
                            contents=messages,
                            config=types.GenerateContentConfig(
                                system_instruction=system_instruction,
                                tools=[nutrition_tools],
                                temperature=0.1
                            )
                        )
        
        # Parse response
        response_text = final_response.text.strip()
        trace.output = response_text[:100] + "..."
        
        try:
            # Clean and parse JSON
            clean_text = response_text.replace("```json", "").replace("```", "").strip()
            data = json.loads(clean_text)
            
            analysis = NutritionAnalysis(
                total_calories=data.get("total_calories", 0),
                total_protein=data.get("total_protein", 0),
                total_carbs=data.get("total_carbs", 0),
                total_fats=data.get("total_fats", 0),
                foods=data.get("foods", [])
            )
            
            trace.status = "completed"
            console.print(f"[green]✓[/green] Analyst: Found {len(analysis.foods)} food items")
            
        except Exception as e:
            trace.status = "error"
            console.print(f"[red]✗[/red] Analyst error: {e}")
            analysis = NutritionAnalysis()
        
        trace.end_time = time.time()
        self.traces.append(trace)
        
        return analysis
    
    def run_coach_agent(
        self, 
        analysis: NutritionAnalysis, 
        profile: UserProfile
    ) -> str:
        """
        AGENT 2: Fitness Coach
        Provides personalized feedback using ADK context management.
        """
        import time
        trace = AgentTrace(agent_name="FitnessCoach", start_time=time.time())
        
        console.print("\n[bold green]💪 Coach Agent: Generating Feedback[/bold green]")
        
        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            console=console,
            transient=True
        ) as progress:
            progress.add_task("Evaluating nutrition plan...", total=None)
            
            system_instruction = f"""You are a Professional Fitness Coach providing evidence-based nutrition guidance.

USER PROFILE:
{json.dumps(profile.to_dict(), indent=2)}

TODAY'S INTAKE:
- Calories: {analysis.total_calories} kcal
- Protein: {analysis.total_protein}g
- Carbs: {analysis.total_carbs}g  
- Fats: {analysis.total_fats}g

Your task:
1. Use calculate_tdee() to determine user's calorie needs
2. Evaluate intake vs. requirements
3. Check protein adequacy (target: 1.6-2.2g/kg for active goals)
4. Provide 2-3 sentences of actionable, motivating feedback
5. Be supportive but honest"""
            
            messages = [
                types.Content(
                    role="user",
                    parts=[types.Part(text="Analyze my nutrition and provide coaching feedback.")]
                )
            ]
            
            response = self.client.models.generate_content(
                model=MODEL_NAME,
                contents=messages,
                config=types.GenerateContentConfig(
                    system_instruction=system_instruction,
                    tools=[nutrition_tools],
                    temperature=0.7
                )
            )
            
            # Handle tool calls
            final_response = response
            for candidate in response.candidates:
                for part in candidate.content.parts:
                    if hasattr(part, 'function_call') and part.function_call:
                        fc = part.function_call
                        trace.tool_calls.append(f"{fc.name}({fc.args})")
                        
                        tool_result = self._execute_tool_call(fc.name, dict(fc.args))
                        
                        messages.append(candidate.content)
                        messages.append(types.Content(
                            role="user",
                            parts=[types.Part(
                                function_response=types.FunctionResponse(
                                    name=fc.name,
                                    response=tool_result
                                )
                            )]
                        ))
                        
                        final_response = self.client.models.generate_content(
                            model=MODEL_NAME,
                            contents=messages,
                            config=types.GenerateContentConfig(
                                system_instruction=system_instruction,
                                temperature=0.7
                            )
                        )
        
        feedback = final_response.text.strip()
        trace.output = feedback[:100] + "..."
        trace.status = "completed"
        trace.end_time = time.time()
        self.traces.append(trace)
        
        console.print(f"[green]✓[/green] Coach: Feedback generated ({len(feedback)} chars)")
        
        return feedback
    
    def display_observability_report(self):
        """Display ADK observability metrics and traces."""
        console.print("\n[bold yellow]═══ ADK OBSERVABILITY REPORT ═══[/bold yellow]\n")
        
        # Summary Table
        table = Table(title="📊 Agent Execution Summary", box=box.ROUNDED)
        table.add_column("Agent", style="cyan")
        table.add_column("Status", style="green")
        table.add_column("Duration (s)", justify="right", style="yellow")
        table.add_column("Tool Calls", justify="right", style="magenta")
        
        for trace in self.traces:
            status_icon = "✓" if trace.status == "completed" else "✗"
            table.add_row(
                trace.agent_name,
                f"{status_icon} {trace.status}",
                f"{trace.duration:.3f}",
                str(len(trace.tool_calls))
            )
        
        console.print(table)
        
        # Trace Tree
        tree = Tree("🔍 [bold]Execution Trace[/bold]")
        
        for trace in self.traces:
            agent_branch = tree.add(f"[cyan]{trace.agent_name}[/cyan] ({trace.duration}s)")
            
            if trace.tool_calls:
                tools_branch = agent_branch.add("[yellow]Tool Calls:[/yellow]")
                for call in trace.tool_calls:
                    tools_branch.add(f"[dim]{call}[/dim]")
            
            if trace.output:
                agent_branch.add(f"[green]Output: {trace.output}[/green]")
        
        console.print("\n")
        console.print(tree)

In [30]:
# VISUALIZATION
# ══════════════════════════════════════════════════════════════════════════

def display_analysis_results(analysis: NutritionAnalysis):
    """Display nutrition analysis with rich formatting."""
    
    summary = (
        f"[bold]Calories:[/bold] {analysis.total_calories} kcal\n"
        f"[bold]Protein:[/bold] {analysis.total_protein}g  |  "
        f"[bold]Carbs:[/bold] {analysis.total_carbs}g  |  "
        f"[bold]Fats:[/bold] {analysis.total_fats}g"
    )
    
    console.print(Panel(
        summary,
        title="📊 Nutritional Summary",
        border_style="cyan",
        box=box.DOUBLE
    ))
    
    if analysis.foods:
        table = Table(title="🍽️ Food Breakdown", box=box.ROUNDED)
        table.add_column("Food", style="cyan")
        table.add_column("Cal", justify="right", style="yellow")
        table.add_column("Protein", justify="right", style="green")
        table.add_column("Carbs", justify="right", style="magenta")
        table.add_column("Fats", justify="right", style="red")
        
        for food in analysis.foods:
            table.add_row(
                food.get('name', 'Unknown'),
                str(food.get('calories', 0)),
                f"{food.get('protein', 0)}g",
                f"{food.get('carbs', 0)}g",
                f"{food.get('fat', 0)}g"
            )
        
        console.print(table)


def display_coach_feedback(feedback: str):
    """Display coach feedback."""
    console.print(Panel(
        f"[bold yellow]{feedback}[/bold yellow]",
        title="💪 Your Coach Says",
        border_style="green",
        box=box.HEAVY
    ))

In [31]:
def main():
    """Main execution flow using ADK."""
    
    # Header
    console.print("\n" + "="*80, style="bold cyan")
    console.print(
        "[bold cyan]GEMINI ADK MULTI-AGENT NUTRITION SYSTEM[/bold cyan]",
        justify="center"
    )
    console.print("[dim]Built with Google Agent Development Kit[/dim]", justify="center")
    console.print("="*80 + "\n", style="bold cyan")
    
    # API Key Check
    if GOOGLE_API_KEY.startswith("PASTE"):
        console.print(Panel(
            "[bold red]⚠️  Please configure your Google API key![/bold red]\n\n"
            "Set GOOGLE_API_KEY at the top of this file.",
            title="Configuration Required",
            border_style="red"
        ))
        return
    
    # Initialize ADK System
    console.print("[bold]Initializing ADK System...[/bold]\n")
    
    adk_system = ADKNutritionSystem(api_key=GOOGLE_API_KEY)
    
    # User Profile
    user_profile = UserProfile(
        weight_kg=80,
        height_cm=180,
        age=35,
        sex="male",
        activity_level="moderate",
        goal="muscle_gain"
    )
    
    console.print(Panel(
        f"Age: {user_profile.age} | Sex: {user_profile.sex.title()} | "
        f"Weight: {user_profile.weight_kg}kg | Height: {user_profile.height_cm}cm\n"
        f"Activity: {user_profile.activity_level.title()} | Goal: {user_profile.goal.replace('_', ' ').title()}",
        title="👤 User Profile",
        border_style="blue"
    ))
    
    # Sample Input
    meal_input = "I ate 200g chicken breast, 2 wheat rotis, 1 banana, and drank a glass of milk"
    
    console.print(f"\n[bold]📝 Meal Input:[/bold] [italic]{meal_input}[/italic]")
    
    # AGENT WORKFLOW
    console.print("\n[bold cyan]═══ SEQUENTIAL AGENT WORKFLOW ═══[/bold cyan]")
    
    # Step 1: Analyst Agent
    analysis = adk_system.run_analyst_agent(meal_input)
    display_analysis_results(analysis)
    
    # Step 2: Coach Agent
    feedback = adk_system.run_coach_agent(analysis, user_profile)
    display_coach_feedback(feedback)
    
    # Step 3: Observability
    adk_system.display_observability_report()
    
    # Footer
    console.print("\n" + "="*80, style="bold cyan")
    console.print(
        "[bold green]✓ ADK Workflow Complete[/bold green]",
        justify="center"
    )
    console.print("="*80 + "\n", style="bold cyan")


if __name__ == "__main__":
    main()

================================================================================

GEMINI ADK MULTI-AGENT NUTRITION SYSTEM

Built with Google Agent Development Kit

================================================================================

Initializing ADK System...

...

                  📊 Agent Execution Summary                  
╭──────────────────┬─────────────┬──────────────┬────────────╮
│ Agent            │ Status      │ Duration (s) │ Tool Calls │
├──────────────────┼─────────────┼──────────────┼────────────┤
│ NutritionAnalyst │ ✓ completed │       10.598 │          4 │
│ FitnessCoach     │ ✓ completed │        4.737 │          1 │
╰──────────────────┴─────────────┴──────────────┴────────────╯

🔍 Execution Trace
├── NutritionAnalyst (10.598s)
│   ├── Tool Calls:
│   │   ├── get_food_nutrition({'food_name': 'chicken breast'})
│   │   ├── get_food_nutrition({'food_name': 'wheat roti'})
│   │   ├── get_food_nutrition({'food_name': 'banana'})
│   │   └── get_food_nutrition({'food_name': 'milk'})
│   └── Output: ```json
│       {
│         "total_calories": 979,
│         "total_protein": 82,
│         "total_carbs": 128,
│         "total_fats": 13,
│       ...
└── FitnessCoach (4.737s)
    ├── Tool Calls:
    │   └── calculate_tdee({'activity_level': 'moderate', 'sex': 'male', 'weight_kg': 80, 'height_cm': 180, 'age': 
    │       35})
    └── Output: Thanks for sharing your intake! Based on your profile (35-year-old male, 80kg, 180cm, moderate 
        activ...

================================================================================

✓ ADK Workflow Complete

================================================================================